In [1]:
import threading
from scapy.all import *
import psutil
import pymysql

In [3]:
class packet_sniffer():
    def __init__(self, sniffing_duration, start_id):
        self.sniffed = scapy.plist.PacketList
        self.sniffed_len = 0
        self.sniffing_duration = sniffing_duration
        self.start_id = start_id
        self.sniffed_info = dict()
        self.sniffed_info['idpackets'] = list()
        self.sniffed_info['timestamp'] = list()
        self.sniffed_info['sip'] = list()
        self.sniffed_info['dip'] = list()
        self.sniffed_info['sport'] = list()
        self.sniffed_info['dport'] = list()
        self.sniffed_info['protocol'] = list()
        self.sniffed_info['process'] = list()
        self.sniffed_info['payload'] = list()
        self.sql = str()
        self.db_rows = str()
        
    def set_data(self, host, user, password, db, charset):
        self.sniffed_len = len(self.sniffed)
        for i in range(self.sniffed_len):
            self.sniffed_info['idpackets'].append(self.start_id)
            self.start_id += 1
            self.sniffed_info['timestamp'].append(self.sniffed[i].time)
            try:
                self.sniffed_info['sip'].append(self.sniffed[i]['IP'].src)
            except IndexError:
                self.sniffed_info['sip'].append('')
            try:
                self.sniffed_info['dip'].append(self.sniffed[i]['IP'].dst)
            except IndexError:
                self.sniffed_info['dip'].append('')
            try:
                self.sniffed_info['sport'].append(str(self.sniffed[i].sport))
            except AttributeError:
                self.sniffed_info['sport'].append('')
            try:
                self.sniffed_info['dport'].append(str(self.sniffed[i].dport))
            except AttributeError:
                self.sniffed_info['dport'].append('')
            try:
                self.sniffed_info['protocol'].append(str(self.sniffed[i].proto))
            except AttributeError:
                self.sniffed_info['protocol'].append('')
            self.sniffed_info['process'].append('')
            self.sniffed_info['payload'].append('')
            
        self.conn = pymysql.connect(host=host, user=user, password=password, db=db, charset=charset)
        self.curs = self.conn.cursor()
        return (self.sniffed_len, [self.sniffed_info['idpackets'], self.sniffed_info['timestamp'], \
                                   self.sniffed_info['sip'], self.sniffed_info['dip'], self.sniffed_info['sport'], \
                                   self.sniffed_info['dport'], self.sniffed_info['protocol'], \
                                   self.sniffed_info['process'], self.sniffed_info['payload']])
        
    def start_sniffing(self):
        try:
            self.sniffed = sniff(timeout=self.sniffing_duration, filter="tcp", prn=lambda x:x.summary())
        except KeyboardInterrupt:
            print('Interrupted')
            #connclose
            sys.exit(0)
            
    def append_process_name_and_payload(self):
        for proc in psutil.process_iter():
            try:
                for dip in self.sniffed_info['dip']:
                    for pcon in proc.connections():
                        dip_value = "raddr=('" + dip + "'"
                        if dip_value in str(pcon):
                            self.sniffed_info['process'][self.sniffed_info['dip'].index(dip)] = proc.name()
                            try:
                                self.sniffed_info['payload'][self.sniffed_info['dip'].index(dip)] = \
                                            str(self.sniffed[self.sniffed_info['dip'].index(dip)]['TCP'].payload)
                            except IndexError:
                                self.sniffed_info['payload'][self.sniffed_info['dip'].index(dip)] = \
                                            str(self.sniffed[self.sniffed_info['dip'].index(dip)]['UDP'].payload)
            except psutil.AccessDenied:
                pass
        print(self.sniffed_info)
                        
    def store_in_db(self, sql, idx, timestamp, sip, dip, sport, dport, protocol, process, payload):
        self.sql = sql
        self.curs.execute(self.sql, (idx, timestamp, sip, dip, sport, dport, protocol, process, payload))
        
        #print(sniffed_info_)
        
    def show_db(self, sql):
        self.conn.commit()
        self.sql = sql
        self.curs.execute(self.sql)
        self.db_rows = self.curs.fetchall()
        return self.db_rows
        
    def close_db(self):
        return self.conn.close()

In [5]:
if __name__ == '__main__':
    sniffer = packet_sniffer(20, 73)
    sniffer.start_sniffing()
    sniffed_len, sniffed_list = sniffer.set_data(host='localhost', user='root', password='', db='testdb', \
                                                  charset='utf8')
    sniffer.append_process_name_and_payload()
    for i in range(sniffed_len):
        sniffer.store_in_db(sql = "insert into packets(idpackets, timestamp, sip, dip, sport, dport, \
                                protocol, process, payload) values (%s, %s, %s, %s, %s, %s, %s, %s, %s)", \
                            idx = sniffed_list[0][i], timestamp = sniffed_list[1][i], sip = sniffed_list[2][i], \
                            dip = sniffed_list[3][i], sport = sniffed_list[4][i], dport = sniffed_list[5][i], \
                            protocol = sniffed_list[6][i], process = sniffed_list[7][i], payload = sniffed_list[8][i])
    print(sniffer.show_db("select * from packets"))
    sniffer.close_db()

Ether / IP / TCP 172.19.87.39:63613 > 176.34.120.1:https A
Ether / IP / TCP 176.34.120.1:https > 172.19.87.39:63613 A
Ether / IP / TCP 172.19.87.39:62473 > 54.169.138.56:https PA / Raw
Ether / IP / TCP 54.169.138.56:https > 172.19.87.39:62473 A
Ether / IP / TCP 54.169.138.56:https > 172.19.87.39:62473 PA / Raw
Ether / IP / TCP 172.19.87.39:62473 > 54.169.138.56:https A
Ether / IP / TCP 172.19.87.39:62505 > 37.252.244.4:5938 PA / Raw
Ether / IP / TCP 37.252.244.4:5938 > 172.19.87.39:62505 PA / Raw
Ether / IP / TCP 172.19.87.39:62505 > 37.252.244.4:5938 A
Ether / IP / TCP 172.19.87.39:63584 > 23.23.97.192:https A
Ether / IP / TCP 23.23.97.192:https > 172.19.87.39:63584 R
Ether / IP / TCP 172.19.87.39:62473 > 54.169.138.56:https PA / Raw
Ether / IP / TCP 54.169.138.56:https > 172.19.87.39:62473 A
Ether / IP / TCP 172.19.87.39:62505 > 37.252.244.4:5938 PA / Raw
Ether / IP / TCP 37.252.244.4:5938 > 172.19.87.39:62505 PA / Raw
Ether / IP / TCP 172.19.87.39:62505 > 37.252.244.4:5938 A
Ether /

{'idpackets': [73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99], 'timestamp': [1494587471, 1494587472, 1494587475, 1494587476, 1494587476, 1494587476, 1494587476, 1494587476, 1494587476, 1494587478, 1494587479, 1494587480, 1494587480, 1494587481, 1494587481, 1494587481, 1494587485, 1494587486, 1494587486, 1494587486, 1494587486, 1494587486, 1494587486, 1494587490, 1494587490, 1494587491, 1494587491], 'sip': ['172.19.87.39', '176.34.120.1', '172.19.87.39', '54.169.138.56', '54.169.138.56', '172.19.87.39', '172.19.87.39', '37.252.244.4', '172.19.87.39', '172.19.87.39', '23.23.97.192', '172.19.87.39', '54.169.138.56', '172.19.87.39', '37.252.244.4', '172.19.87.39', '172.19.87.39', '54.169.138.56', '54.169.138.56', '172.19.87.39', '172.19.87.39', '37.252.244.4', '172.19.87.39', '172.19.87.39', '216.58.197.133', '172.19.87.39', '37.252.244.4'], 'dip': ['176.34.120.1', '172.19.87.39', '54.169.138.56', '172.19.87.39', '172.19.87.39', 